In [8]:
import os
import pandas as pd
import re


log_directory = 'smest_logs/'

# 存储结果的列表
results = []

# 1. 获取目录下所有日志文件
log_files = [f for f in os.listdir(log_directory) if f.endswith('.logs')]


# 2. 处理每个日志文件
for file_name in log_files:
    # 解析文件名
    name_parts = file_name.replace('.logs', '').split('-')
    
    # 读取日志内容
    with open(os.path.join(log_directory, file_name), 'r', encoding='utf-8') as f:
        content = f.read()
        
    # 3. 检查内容
    if 'torch.cuda.OutOfMemoryError: CUDA out of memory' in content:
        value = 'OOM'
    else:
        # 查找 train_samples_per_second 的值
        # match = re.search(r'train_samples_per_second\s*=\s*(\d+\.\d+)', content)
        # match = re.search(r'50%\|█.*?(\d+\.\d+)s/it\]', content)
        match = re.search(r'50%\|█.*?6/12.*?(\d+\.\d+)s/it\]', content)
        value = float(match.group(1)) if match else None
    
    # 组合结果
    result = {
        'inst_cnt': name_parts[1].replace('inst',''),
        'zero_stage': name_parts[2],
        'is_offload': 'Y' if len(name_parts) == 6 else 'N',
        'micro_bs': name_parts[4]if len(name_parts) == 6 else name_parts[3],
        'accum_steps': name_parts[5] if len(name_parts) == 6 else name_parts[4],
        'speed': value
    }
    results.append(result)

# 创建 DataFrame
df = pd.DataFrame(results)

df


,inst_cnt,zero_stage,is_offload,micro_bs,accum_steps,speed
0,1,z1,N,mbs2,acm4,2.85
1,2,z1,N,mbs2,acm2,1.82
2,1,z1,N,mbs1,acm8,2.49
3,2,z3,N,mbs2,acm2,2.77
4,1,z3,N,mbs2,acm4,4.74
5,1,z2,N,mbs4,acm2,OOM
6,1,z3,Y,mbs2,acm4,7.17
7,1,z2,Y,mbs2,acm4,7.57
8,1,z2,Y,mbs4,acm2,6.57
9,2,z3,Y,mbs2,acm2,3.82


In [5]:
df.to_csv('p5-train-perf-3.csv', index=False)